In [1]:
import pymysql

import os
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from operator import itemgetter

In [2]:
match_id_list = pd.read_csv('data/match_id_list.csv', usecols=['match_id'], low_memory=False)
match_id_list = match_id_list['match_id']
len(match_id_list)

60209

## 데이터 경로

In [3]:
timelineDir = 'data/match_timeline/'
matchDir = 'data/match/'

## DB 연결

In [4]:
conn = pymysql.connect(
    host="localhost",
    user='root',
    password='240812'
)

#conn.autocommit(False)
cur = conn.cursor()
cur.execute('use LOL_data;')

0

## 건님 코드

In [5]:
building_kill = 'BUILDING_KILL'
ward_placed = 'WARD_PLACED'
elite_monster_kill = 'ELITE_MONSTER_KILL'
turret_plate_destroyed = 'TURRET_PLATE_DESTROYED'
objective_bounty_prestart = 'OBJECTIVE_BOUNTY_PRESTART'
champion_kill = 'CHAMPION_KILL'
champion_special_kill = 'CHAMPION_SPECIAL_KILL'
item_destroyed = 'ITEM_DESTROYED'
item_purchased = 'ITEM_PURCHASED'
item_sold = "ITEM_SOLD"
game_end = "GAME_END"
level_up = "LEVEL_UP"
skill_level_up = "SKILL_LEVEL_UP"
ward_kill =  "WARD_KILL"
item_undo = "ITEM_UNDO"



In [6]:
def assist_id_list2int(data): # assist ID의 리스트를 integer로 변환
    data = [1 if x in data else 0 for x in range(11)]
    return int(''.join(map(str, data)), 2)

def get_assist_id_list(data): # integer를 assist ID의 리스트로 변환
    data = ''.join(['0' for x in range(11 - len(bin(data)[2:]))]) + bin(data)[2:]
    return [i for i in range(len(data)) if data[i] == '1']

def query_insert_buildingKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    if 'towerType' not in event.keys(): event['towerType'] = 'null'
        
    return f'''insert into {building_kill} (match_id, assist_id, bounty, building_type, participant_id, line_type, team_id, timestamp, tower_type)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, '{event['buildingType']}', {event['killerId']}, '{event['laneType']}',
            {event['teamId']}, {event['timestamp']}, '{event['towerType']}');'''

def query_insert_wardPlaced(event, matchID):
    return f'''insert into {ward_placed} (match_id, participant_id, timestamp, ward_type)
            values ({matchID}, {event['creatorId']}, {event['timestamp']}, '{event['wardType']}');'''

def query_insert_eliteMonsterKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    if 'monsterSubType' not in event.keys():
        event['monsterSubType'] = 'null'

    return f'''insert into {elite_monster_kill} (match_id, assist_id, bounty, participant_id, monster_subtype, monster_type, timestamp)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, {event['killerId']}, '{event['monsterSubType']}', '{event['monsterType']}',
            {event['timestamp']});'''

def query_insert_turretPlateDestroyed(event, matchID):
    return f'''insert into {turret_plate_destroyed} (match_id, participant_id, line_type, team_id, timestamp)
            values ({matchID}, {event['killerId']}, '{event['laneType']}', {event['teamId']}, {event['timestamp']});'''

def query_insert_objectiveBountyPrestart(event, matchID):
    return f'''insert into {objective_bounty_prestart} (match_id, actual_starttime, team_id, timestamp)
            values ({matchID}, {event['actualStartTime']}, {event['teamId']}, {event['timestamp']});'''

def query_insert_championKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    return f'''insert into {champion_kill} (match_id, assist_id, bounty, killstreak_length, participant_id, shutdown_bounty, timestamp, victim_id)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, {event['killStreakLength']}, {event['killerId']}, {event['shutdownBounty']},
            {event['timestamp']}, {event['victimId']});'''

def query_insert_championSpecialKill(event, matchID):
    if 'multiKillLength' not in event.keys() or event['multiKillLength'] == None:
        event['multiKillLength'] = 'null'
        
    return f'''insert into {champion_special_kill} (match_id, kill_type, participant_id, multikil_length, timestamp)
            values ({matchID}, '{event['killType']}', {event['killerId']}, {event['multiKillLength']}, {event['timestamp']});'''

def query_insert_itemDestroyed(event, matchID):
    return f'''insert into {item_destroyed} (match_id, item_id, participant_id, timestamp)
            values ({matchID}, {event['itemId']}, {event['participantId']}, {event['timestamp']});'''

def query_insert_itemPurchased(event, matchID):
    return f'''insert into {item_purchased} (match_id, item_id, participant_id, timestamp)
            values ({matchID}, {event['itemId']}, {event['participantId']}, {event['timestamp']});'''



## 수진님 코드

In [7]:
champstatscols = {
    'abilityHaste': 'abilityHaste',
    'abilityPower': 'abilityPower',
    'armor': 'armor',
    'armorPen': 'armorpen',
    'armorPenPercent': 'armorpen_percent',
    'attackDamage': 'attackdamage',
    'attackSpeed': 'attackspeed',
    'bonusArmorPenPercent': 'bonus_app',
    'bonusMagicPenPercent': 'bonus_mpp',
    'ccReduction': 'cc_reduction',
    'cooldownReduction': 'cd_reduction',
    'health': 'health',
    'healthMax': 'health_max',
    'healthRegen': 'health_regen',
    'lifesteal': 'lifesteal',
    'magicPen': 'magicpen',
    'magicPenPercent': 'magicpenpercent',
    'magicResist': 'magicresist',
    'movementSpeed': 'movement_speed',
    'omnivamp': 'omnivamp',
    'physicalVamp': 'physicalvamp',
    'power': 'power',
    'powerMax': 'power_max',
    'powerRegen': 'power_regen',
    'spellVamp': 'spellvamp',
    'participantId': 'participant_id',
    'currentGold': 'current_gold',
    'magicDamageDone': 'magic__damage_done',
    'magicDamageDoneToChampions': 'mdd_to_champion',
    'magicDamageTaken': 'magic_damage_taken',
    'physicalDamageDone': 'physic_damage_done',
    'physicalDamageDoneToChampions': 'pdd_to_champion',
    'physicalDamageTaken': 'physic_damage_taken',
    'totalDamageDone': 'toal_damage_done',
    'totalDamageDoneToChampions': 'tdd_to_champion',
    'totalDamageTaken': 'total_damage_taken',
    'trueDamageDone': 'true_damage_done',
    'trueDamageDoneToChampions': 'truedamage_to_champion',
    'trueDamageTaken': 'true_damage_taken',
    'goldPerSecond': 'gold_per_second',
    'jungleMinionKilled': 'jungle_killed',
    'level': 'level',
    'minionKilled': 'minion_killed',
    'timeEnemySpendControlled': 'time_enemy_controlled',
    'totalGold': 'total_gold',
    'xp': 'xp',
    'timestamp': 'timestamp'
}

In [8]:
def getChampionData(matchData):
    championList = list()
    matchId = matchData['metadata']['matchId'] 
    matchId = matchId.split('_')[1]
    for championData in matchData['info']['participants']:
        values = itemgetter(*['participantId', 'championId'])(championData)
        championList.append((matchId,)+ values  )

    cols = ['match_id', 'participant_id', 'champion_id']
    championDataDf= pd.DataFrame(championList, columns = cols, ) # ['matchId', 'participantId', 'championId']
    championDataDf.to_sql('champion_participant_id', con=cur, if_exists='append', index=False)

    return matchId

def getChampionStats(infoFrameData, matchId):
    resultFrame = []
    for i in range(0, len(infoFrameData),1):
        for pN in range(1, 9, 1):
            resultFrame.append({**infoFrameData[i]['participantFrames'][str(pN)]['championStats'],\
                       'participantId' : infoFrameData[i]['participantFrames'][str(pN)]['participantId'],\
                       'currentGold': infoFrameData[i]['participantFrames'][str(pN)]['currentGold'],\
                        **infoFrameData[i]['participantFrames'][str(pN)]['damageStats'],\
                        'goldPerSecond':infoFrameData[i]['participantFrames'][str(pN)]['goldPerSecond'],\
                        'jungleMinionKilled': infoFrameData[i]['participantFrames'][str(pN)]['jungleMinionsKilled'],\
                        'level': infoFrameData[i]['participantFrames'][str(pN)]['level'],\
                        'minionKilled': infoFrameData[i]['participantFrames'][str(pN)]['minionsKilled'],\
                        'timeEnemySpendControlled':infoFrameData[i]['participantFrames'][str(pN)]['timeEnemySpentControlled'],\
                        'totalGold':infoFrameData[i]['participantFrames'][str(pN)]['totalGold'],\
                        'xp':infoFrameData[i]['participantFrames'][str(pN)]['xp'], 'timestamp': i},)
    
    resultFrameDf = pd.DataFrame(resultFrame).rename(columns=champstatscols)
    resultFrameDf['match_id'] = matchId

    resultFrameDf.to_sql('champion_stat_per_timestamp', con=conn, if_exists='append', index=False)

def convertToDf(bucketData,  matchId):
    
    bucketDf = pd.DataFrame(bucketData)
    
    bucketDf['matchId'] = int(matchId)

    if 'type' in bucketDf.columns:
        bucketDf = bucketDf.drop(columns=['type'])

    return bucketDf

def getgameLogData(matchLineData, matchId, engine):
    
    itemSoldBucket = list()
    gameEndBucket = list()
    levelUpBucket = list()
    skilLevelUpBucket = list()
    wardKillBucket = list()

    for i in range(0, len(matchLineData),1):
        elementData = matchLineData[i]['events']

        for e in elementData : 
                if e['type'] == "ITEM_SOLD":
                        itemSoldBucket.append(e)
    
                if e['type'] == "GAME_END":
                        gameEndBucket.append(e)
    
                if e['type'] == "LEVEL_UP":
                        levelUpBucket.append(e)

                if e['type'] == "SKILL_LEVEL_UP":
                        skilLevelUpBucket.append(e)
    
                if e['type'] ==  "WARD_KILL":
                        wardKillBucket.append(e)
    



    itemSoldDf = convertToDf(itemSoldBucket, matchId).drop('type', axis=1)
    gameEndDf = convertToDf(gameEndBucket, matchId).drop(['type', 'gameId'], axis=1)
    levelUpDf = convertToDf(levelUpBucket, matchId).drop('type', axis=1)
    skillLevelUpDf = convertToDf(skilLevelUpBucket, matchId).drop('type', axis=1)
    wardKillDf = convertToDf(wardKillBucket, matchId).drop('type', axis=1)

    itemSoldDf.columns = ['item_id', 'participant_id', 'timestamp', 'match_id']
    gameEndDf.columns = ['real_timestamp', 'timestamp', 'winning_team', 'match_id']
    levelUpDf.columns = ['level', 'participant_id', 'timestamp', 'match_id']
    skillLevelUpDf.columns = ['levelup_type', 'participant_id', 'skill_slot', 'timestamp', 'match_id']
    wardKillDf.columns = ['participant_id', 'timestamp', 'ward_type', 'match_id']

    itemSoldDf.to_sql(item_sold, con=engine, if_exists='append', index=False)
    gameEndDf.to_sql(game_end, con=engine, if_exists='append', index=False)
    levelUpDf.to_sql(level_up, con=engine, if_exists='append', index=False)
    skillLevelUpDf.to_sql(skill_level_up, con=engine, if_exists='append', index=False)
    wardKillDf.to_sql(ward_kill, con=engine, if_exists='append', index=False)


    #각각 csv로 변환

    # itemSoldDf.to_csv(os.path.join(fileOutputDir, f'{matchId}_ITEM_SOLD.csv'))
    # gameEndDf.to_csv( os.path.join(fileOutputDir, f'{matchId}_GAME_END.csv'))
    # levelUpDf.to_csv( os.path.join(fileOutputDir, f'{matchId}_LEVEL_UP.csv'))
    # skillLevelUpDf.to_csv( os.path.join(fileOutputDir, f'{matchId}_SKILL_LEVEL_UP.csv'))
    # wardKillDf.to_csv( os.path.join(fileOutputDir, f'{matchId}_WARD_KILL.csv'))

    #return  itemSoldDf, gameEndDf, levelUpDf, skillLevelUpDf, wardKillDf

## 창길 코드

수진님 코드 수정

In [9]:
def insert_championParticipantID(matchData, matchID, cur):
    for (idx, championData) in enumerate(matchData['info']['participants']):
        champID = championData['championId']
        query = f'''insert into champion_participant_id (match_id, participant_id, champion_id) values ({matchID}, {idx+1}, {champID});'''
        cur.execute(query)

def insert_championStats(frameData, matchID,cur):
    for pN in range(1, 11, 1):
        query = f'''insert into champion_stat_per_timestamp 
                    (match_id, participant_id, timestamp,
                     abilityHaste, abilityPower,
                     armor, armorpen, armorpen_percent, attackdamage, attackspeed, 
                     bonus_app, bonus_mpp, cc_reduction, cd_reduction, 
                     health, health_max, health_regen, lifesteal, magicpen, magicpenpercent, magicresist, movement_speed, 
                     omnivamp, physicalvamp, power, power_max, power_regen, spellvamp, current_gold, 
                     magic__damage_done, mdd_to_champion, magic_damage_taken, 
                     physic_damage_done, pdd_to_champion, physic_damage_taken, 
                     toal_damage_done, tdd_to_champion, total_damage_taken, 
                     true_damage_done, truedamage_to_champion, true_damage_taken, 
                     gold_per_second, jungle_killed, level, minion_killed, time_enemy_controlled, total_gold, xp) 
                     values (
                    {matchID}, 
                    {pN},
                    {frameData['timestamp']},
                    {frameData['participantFrames'][str(pN)]['championStats']['abilityHaste']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['abilityPower']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['armor']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['armorPen']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['armorPenPercent']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['attackDamage']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['attackSpeed']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['bonusArmorPenPercent']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['bonusMagicPenPercent']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['ccReduction']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['cooldownReduction']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['health']}, 
                    {frameData['participantFrames'][str(pN)]['championStats']['healthMax']},
                    {frameData['participantFrames'][str(pN)]['championStats']['healthRegen']},
                    {frameData['participantFrames'][str(pN)]['championStats']['lifesteal']},
                    {frameData['participantFrames'][str(pN)]['championStats']['magicPen']},
                    {frameData['participantFrames'][str(pN)]['championStats']['magicPenPercent']},
                    {frameData['participantFrames'][str(pN)]['championStats']['magicResist']},
                    {frameData['participantFrames'][str(pN)]['championStats']['movementSpeed']},
                    {frameData['participantFrames'][str(pN)]['championStats']['omnivamp']},
                    {frameData['participantFrames'][str(pN)]['championStats']['physicalVamp']},
                    {frameData['participantFrames'][str(pN)]['championStats']['power']},
                    {frameData['participantFrames'][str(pN)]['championStats']['powerMax']},
                    {frameData['participantFrames'][str(pN)]['championStats']['powerRegen']},
                    {frameData['participantFrames'][str(pN)]['championStats']['spellVamp']},
                    {frameData['participantFrames'][str(pN)]['currentGold']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['magicDamageDone']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['magicDamageDoneToChampions']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['magicDamageTaken']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['physicalDamageDone']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['physicalDamageDoneToChampions']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['physicalDamageTaken']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['totalDamageDone']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['totalDamageDoneToChampions']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['totalDamageTaken']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['trueDamageDone']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['trueDamageDoneToChampions']},
                    {frameData['participantFrames'][str(pN)]['damageStats']['trueDamageTaken']},
                    {frameData['participantFrames'][str(pN)]['goldPerSecond']},
                    {frameData['participantFrames'][str(pN)]['jungleMinionsKilled']},
                    {frameData['participantFrames'][str(pN)]['level']},
                    {frameData['participantFrames'][str(pN)]['minionsKilled']},
                    {frameData['participantFrames'][str(pN)]['timeEnemySpentControlled']},
                    {frameData['participantFrames'][str(pN)]['totalGold']},
                    {frameData['participantFrames'][str(pN)]['xp']});'''
        cur.execute(query)


In [10]:
def query_insert_gameEnd(event, matchID):
    return f'''insert into {game_end} (match_id, real_timestamp, timestamp, winning_team)
            values ({matchID}, {event['realTimestamp']}, {event['timestamp']}, {event['winningTeam']});'''
def query_insert_itemSold(event, matchID):
    return f'''insert into {item_sold} (match_id, item_id, participant_id, timestamp)
            values ({matchID}, {event['itemId']}, {event['participantId']}, {event['timestamp']});'''
def query_insert_levelUp(event, matchID):
    return f'''insert into {level_up} (match_id, level, participant_id, timestamp)
            values ({matchID}, {event['level']}, {event['participantId']}, {event['timestamp']});'''
def query_insert_skillLevelUp(event, matchID):
    return f'''insert into {skill_level_up} (match_id, levelup_type, participant_id, skill_slot, timestamp)
            values ({matchID}, '{event['levelUpType']}', {event['participantId']}, {event['skillSlot']}, {event['timestamp']});'''
def query_insert_wardKill(event, matchID):
    return f'''insert into {ward_kill} (match_id, participant_id, timestamp, ward_type)
            values ({matchID}, {event['killerId']}, {event['timestamp']}, '{event['wardType']}');'''

건님코드 수정

In [11]:
event_insert_func_dict = {
    building_kill: query_insert_buildingKill, #
    turret_plate_destroyed: query_insert_turretPlateDestroyed, #
    ward_placed: query_insert_wardPlaced, #
    ward_kill: query_insert_wardKill, #
    elite_monster_kill: query_insert_eliteMonsterKill, #
    objective_bounty_prestart: query_insert_objectiveBountyPrestart, #
    champion_kill: query_insert_championKill, #
    champion_special_kill: query_insert_championSpecialKill, #
    item_destroyed: query_insert_itemDestroyed, 
    item_purchased: query_insert_itemPurchased,
    item_sold: query_insert_itemSold,
    level_up: query_insert_levelUp,
    skill_level_up: query_insert_skillLevelUp,
    game_end: query_insert_gameEnd
}
def insert_events(match, match_tl, cur):
    # match id 추출
    matchID = int(match_tl['metadata']['matchId'].split('_')[-1])
    
    # 참가자 id, 챔피언 id 삽입
    insert_championParticipantID(match, matchID, cur)

    for frame in match_tl['info']['frames']:
        # 이벤트 삽입
        for event in frame['events']:
            # print(event['type'])
            if event['type'] in event_insert_func_dict.keys():
                query = event_insert_func_dict[event['type']](event, matchID)
            else:
                #print(f'Unknown event type: {event["type"]}')
                query = None

            if query:
                #print(query)
                cur.execute(query)
        # 챔피언 스펙 삽입
        insert_championStats(frame, matchID, cur)

솔로랭크 매치인지 확인

In [12]:
# find solo rank 5x5 match
def isSoloRank(match):
    return match['info']['queueId'] == 420

In [13]:
## test
# read match file
match = json.load(open(matchDir+match_id_list[0]+'.json', 'r'))
match_tl = json.load(open(timelineDir+match_id_list[0]+'.json', 'r'))

print(match_id_list[0], isSoloRank(match))

KR_7346623650 True


삽입 테스트

In [14]:
insert_events(match, match_tl, cur)

## 실행코드

In [15]:
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

In [ ]:
now = datetime.now()
formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
print("시작 시각:", formatted_time)

# cnt = 0
regame = []
nosolo = []
exceps = []
for e in tqdm(list(match_id_list)):
    try:
        matchPath = matchDir + e + '.json'
        matchTimelineDir = timelineDir + e + '.json'

        # 파일 있는지 확인
        if os.path.isfile(matchPath) and os.path.isfile(matchTimelineDir):
            with open( matchTimelineDir ) as f:
                match_tl = json.load(f)
            with open( matchPath ) as f:
                match = json.load(f)
            
            if not isSoloRank(match):
                print('not SoloRank :', e)
                nosolo.append(e)
                continue
    
            # 5분 미만 게임 제외
            if len(match_tl['info']['frames']) < 5:
                print('다시하기 ', e)
                regame.append(e)
                continue
            
            insert_events(match,match_tl, cur)
            conn.commit()
        else: 
            continue
    except Exception as err:
        print('error ', e, err)
        exceps.append((e, err))
        continue

now2 = datetime.now()

formatted_time = now2.strftime("%Y-%m-%d %H:%M:%S")
print("마무리 시각:", formatted_time)
print(f"소요 시간: 약 {(now2 - now).total_seconds() / 60}분")
print(f'regame count: {len(regame)} / exception count: {len(exceps)}')

시작 시각: 2024-11-07 12:56:04


  0%|          | 0/60209 [00:00<?, ?it/s]